# ArionHash R1CS & Plonk Constraints

In this SageMath notebook we compute the number of R1CS and Plonk constraints for ArionHash and compare them to Anemoi, Griffin and Poseidon.

The number of constraints for Arion is computed via
\begin{align*}
    N_\textsf{R1CS} &= r \cdot \big( \left( n - 1 \right) \cdot \left( d_{1, \text{inc}} + 2 \right) + d_{2, \text{inc}} \big) \\
    N_{\textsf{Plonk}, 2} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 6) + d_{2, inc} - 1 \big) +
    \begin{dcases}
        (r + 1) \cdot n \cdot (n - 1), & n = 2, 3 \\
        (r + 1) \cdot 4 \cdot (n - 1), & n \geq 4,
    \end{dcases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= r \cdot \big( (n - 1) \cdot (d_{1, inc} + 4) + d_{2, inc} \big) +
    \begin{dcases}
        (r + 1) \cdot n, &n = 2, 3, \\
        (r + 1) \cdot \left( n + 2 + \left\lceil \frac{n - 3}{2} \right\rceil + \left\lceil \frac{n - 4}{2} \right\rceil \right), & n \geq 4,
    \end{dcases}
\end{align*}

In [1]:
def arion_R1CS_constraints(d_1, d_2, n, r):
    if d_1 == 3:
        d_1_inc = 2
    elif d_1 == 5:
        d_1_inc = 3
    else:
        print("Exponent", d_1, "not implemented.")
        return
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        print("Exponent", d_2, "not implemented.")
        return
    return r * ((n - 1) * (d_1_inc + 2) + d_2_inc)

def arion_Plonk_2_wire_constraints(d_1, d_2, n, r):
    if d_1 == 3:
        d_1_inc = 2
    elif d_1 == 5:
        d_1_inc = 3
    else:
        print("Exponent", d_1, "not implemented.")
        return
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        print("Exponent", d_2, "not implemented.")
        return
    return r * ((n - 1) * (d_1_inc + 10) + d_2_inc - 1) + 4 * (n - 1)

def arion_Plonk_3_wire_constraints(d_1, d_2, n, r):
    if d_1 == 3:
        d_1_inc = 2
    elif d_1 == 5:
        d_1_inc = 3
    else:
        print("Exponent", d_1, "not implemented.")
        return
    if d_2 in [121, 123, 125, 161, 257]:
        d_2_inc = 9
    else:
        print("Exponent", d_2, "not implemented.")
        return
    if n == 2:
        return r * ((n - 1) * (d_1_inc + 4) + d_2_inc + n) + n
    elif n == 3:
        return r * ((n - 1) * (d_1_inc + 4) + d_2_inc + n + ceil((n - 3) / 2)) + n + ceil((n - 3) / 2)
    else:
        return r * ((n - 1) * (d_1_inc + 5) + d_2_inc + 3 + ceil((n - 3) / 2) + ceil((n - 4) / 2)) + n + 2 + ceil((n - 3) / 2) + ceil((n - 4) / 2)

The number of constraints for Griffin is computed via
\begin{align*}
    N_\textsf{R1CS} &= 2 \cdot r \cdot \left( d_{inc} + n - 2 \right) \\
    N_{\textsf{Plonk}, 2} 
    &= r \cdot (4 \cdot n - 5) + (r + 1) \cdot
    \begin{dcases}
        5, & n = 3, \\
        11, & n = 4, \\
        30, & n = 8, \\
        \frac{11 \cdot n}{4} + 2 \cdot n - 4, & n \geq 12,
    \end{dcases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= r \cdot (4 \cdot n - 3) + (r + 1) \cdot
    \begin{dcases}
        3, & n = 3, \\
        6, & n = 4, \\
        20, & n = 8, \\
        \frac{6 \cdot n}{4} + 4 \cdot \left\lfloor \frac{\frac{n}{4} - 1}{2} \right\rfloor + n, & n \geq 12.
    \end{dcases}
\end{align*}

In [2]:
def griffin_R1CS_constraints(d, n, r):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    return 2 * r * (d_inc + n - 2)

def griffin_Plonk_2_wire_constraints(d, n, r):
    if d == 3:
        N = r * (4 * n - 5)
    elif d == 5:
        N = r * (4 * n - 3)
    else:
        print("Exponent", d, "not implemented.")
        return
    if n == 3:
        return N + (r + 1) * 5
    elif n == 4:
        return N + (r + 1) * 11
    elif n == 8:
        return N + (r + 1) * 30
    elif n % 4 == 0:
        return N + (r + 1) * (11 * n / 4 + 2 * n - 4)
    else:
        print("Branch number", n, "not possible for Griffin.")
        return

def griffin_Plonk_3_wire_constraints(d, n, r):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    N = r * (2 * d_inc + 3 * n - 8)
    if n == 3:
        return N + (r + 1) * 3
    elif n == 4:
        return N + (r + 1) * 6
    elif n == 8:
        return N + (r + 1) * 20
    elif n % 4 == 0:
        return N + (r + 1) * (6 * n / 4 * floor((n / 4 - 1) / 2) + n)
    else:
        print("Branch number", n, "not possible for Griffin.")
        return

The number of constraints for Anemoi is computed via
\begin{align*}
    N_\textsf{R1CS} &= \frac{r \cdot n}{2} \cdot \left( d_{inc} + 2\right) \\
    N_{\textsf{Plonk}, 2} 
    &= \frac{r \cdot n}{2} \cdot (d_{inc} + 5) + (r + 1) \cdot
    \begin{dcases}
        2, & n = 2, \\
        n \cdot \left( \frac{n}{2} - 1 \right), & n = 4, \\
        10, & n = 6, \\
        16, & n = 8,
    \end{dcases}
    \\
    N_{\textsf{Plonk}, 3} 
    &= \frac{r \cdot n}{2} \cdot (d_{inc} + 3) + (r + 1) \cdot
    \begin{dcases}
        (r + 1) \cdot n, & n = 2, 4, \\
        6, & n = 6, \\
        12, & n = 8.
    \end{dcases}
\end{align*}

In [3]:
def anemoi_R1CS_constraints(d, n, r):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    return r * n / 2 * (d_inc + 2)

def anemoi_Plonk_2_wire_constraints(d, n, r):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    N = r * n / 2 * (d_inc + 5)
    if n == 2:
        return N + (r + 1) * n
    elif n == 4:
        return N + (r + 1) * n * (n / 2 - 1)
    elif n == 6:
        return N + (r + 1) * 10
    elif n == 8:
        return N + (r + 1) * 16
    else:
        print("Branch number", n, "not implemented for Anemoi.")
        return

def anemoi_Plonk_3_wire_constraints(d, n, r):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    N = r * n / 2 * (d_inc + 3)
    if n == 2:
        return N + (r + 1) * n
    elif n == 4:
        return N + (r + 1) * n
    elif n == 6:
        return N + (r + 1) * 6
    elif n == 8:
        return N + (r + 1) * 12
    else:
        print("Branch number", n, "not implemented for Anemoi.")
        return

The number of constraints for Poseidon is computed via
\begin{align*}
    N_\textsf{R1CS} &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) \\
    N_{\textsf{Plonk}, 2} 
    &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + (r + 1) \cdot n \cdot (n - 1) \\
    N_{\textsf{Plonk}, 3} 
    &= d_{inc} \cdot \left( n \cdot r_f + r_p \right) + (r + 1) \cdot n \cdot
    \begin{dcases}
        1, & n = 2,3, \\
        \left\lceil \frac{n - 3}{2} \right\rceil, & n \geq 4.
    \end{dcases}
\end{align*}

In [4]:
def poseidon_R1CS_constraints(d, n, r_f, r_p):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    return d_inc * (n * r_f + r_p)

def poseidon_Plonk_2_wire_constraints(d, n, r_f, r_p):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    return d_inc * (n * r_f + r_p) + (r_f + r_p) * n * (n - 1)

def poseidon_Plonk_3_wire_constraints(d, n, r_f, r_p):
    if d == 3:
        d_inc = 2
    elif d == 5:
        d_inc = 3
    else:
        print("Exponent", d, "not implemented.")
        return
    N = d_inc * (n * r_f + r_p)
    if n == 2:
        return N + (r_f + r_p) * n
    elif n == 3:
        return N + (r_f + r_p) * n
    else:
        return N + (r_f + r_p) * n * ceil((n - 3) / 2)

## ArionHash

In [5]:
arion_hash_params = [
#[d_1, d_2, n, r]
 [3, 257, 3, 6],
 [5, 257, 3, 6],
 [3, 257, 4, 6],
 [5, 257, 4, 5],
 [3, 257, 5, 5],
 [5, 257, 5, 5],
 [3, 257, 6, 5],
 [5, 257, 6, 5],
 [3, 257, 8, 4],
 [5, 257, 8, 4],
]

print("ArionHash constraints")
print("d_1", "\t", 
      "d_2", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in arion_hash_params:
    d_1 = param[0]
    d_2 = param[1]
    n = param[2]
    r = param[3]
    print(d_1, "\t",
          d_2, "\t", 
          n, "\t", 
          r, "\t", 
          arion_R1CS_constraints(d_1, d_2, n, r), "\t\t", 
          arion_Plonk_2_wire_constraints(d_1, d_2, n, r), "\t\t\t", 
          arion_Plonk_3_wire_constraints(d_1, d_2, n, r))

ArionHash constraints
d_1 	 d_2 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 257 	 3 	 6 	 102 		 200 			 147
5 	 257 	 3 	 6 	 114 		 212 			 159
3 	 257 	 4 	 6 	 126 		 276 			 211
5 	 257 	 4 	 5 	 120 		 247 			 192
3 	 257 	 5 	 5 	 125 		 296 			 219
5 	 257 	 5 	 5 	 145 		 316 			 239
3 	 257 	 6 	 5 	 145 		 360 			 261
5 	 257 	 6 	 5 	 170 		 385 			 286
3 	 257 	 8 	 4 	 148 		 396 			 279
5 	 257 	 8 	 4 	 176 		 424 			 307


In [6]:
aggressive_arion_hash_params = [
#[d, n, r]
 [3, 257, 3, 5],
 [5, 257, 3, 4],
 [3, 257, 4, 4],
 [5, 257, 4, 4],
 [3, 257, 5, 4],
 [5, 257, 5, 4],
 [3, 257, 6, 4],
 [5, 257, 6, 4],
 [3, 257, 8, 4],
 [5, 257, 8, 4],
]

print("Aggressive Hash constraints")
print("d_1", "\t", 
      "d_2", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in aggressive_arion_hash_params:
    d_1 = param[0]
    d_2 = param[1]
    n = param[2]
    r = param[3]
    print(d_1, "\t",
          d_2, "\t", 
          n, "\t", 
          r, "\t", 
          arion_R1CS_constraints(d_1, d_2, n, r), "\t\t", 
          arion_Plonk_2_wire_constraints(d_1, d_2, n, r), "\t\t\t", 
          arion_Plonk_3_wire_constraints(d_1, d_2, n, r))

Aggressive Hash constraints
d_1 	 d_2 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 257 	 3 	 5 	 85 		 168 			 123
5 	 257 	 3 	 4 	 76 		 144 			 107
3 	 257 	 4 	 4 	 84 		 188 			 143
5 	 257 	 4 	 4 	 96 		 200 			 155
3 	 257 	 5 	 4 	 100 		 240 			 177
5 	 257 	 5 	 4 	 116 		 256 			 193
3 	 257 	 6 	 4 	 116 		 292 			 211
5 	 257 	 6 	 4 	 136 		 312 			 231
3 	 257 	 8 	 4 	 148 		 396 			 279
5 	 257 	 8 	 4 	 176 		 424 			 307


## Anemoi

In [7]:
anemoi_params = [
#[d, n, r]
 [3, 4, 12],
 [5, 4, 12],
 [3, 6, 10],
 [5, 6, 10],
 [3, 8, 10],
 [5, 8, 10],
]

print("Anemoi constraints")
print("d", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t",
      "N_Plonk_3_wire")
for param in anemoi_params:
    d = param[0]
    n = param[1]
    r = param[2]
    print(d, "\t",
          n, "\t", 
          r, "\t", 
          anemoi_R1CS_constraints(d, n, r), "\t\t",
          anemoi_Plonk_2_wire_constraints(d, n, r), "\t\t\t",
             anemoi_Plonk_3_wire_constraints(d, n, r))

Anemoi constraints
d 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 4 	 12 	 96 		 220 			 172
5 	 4 	 12 	 120 		 244 			 196
3 	 6 	 10 	 120 		 320 			 216
5 	 6 	 10 	 150 		 350 			 246
3 	 8 	 10 	 160 		 456 			 332
5 	 8 	 10 	 200 		 496 			 372


## Griffin

In [8]:
griffin_params = [
#[d, n, r]
 [3, 3, 16],
 [5, 3, 12],
 [3, 4, 14],
 [5, 4, 11],
 [3, 8, 11],
 [5, 8, 9],
]

print("Griffin constraints")
print("d", "\t", 
      "n", "\t", 
      "r", "\t", 
      "N_R1CS", "\t", 
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in griffin_params:
    d = param[0]
    n = param[1]
    r = param[2]
    print(d, "\t", 
          n, "\t", 
          r, "\t", 
          griffin_R1CS_constraints(d, n, r), "\t\t",
          griffin_Plonk_2_wire_constraints(d, n, r), "\t\t\t",
          griffin_Plonk_3_wire_constraints(d, n, r))

Griffin constraints
d 	 n 	 r 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 3 	 16 	 96 		 197 			 131
5 	 3 	 12 	 96 		 173 			 123
3 	 4 	 14 	 112 		 319 			 202
5 	 4 	 11 	 110 		 275 			 182
3 	 8 	 11 	 176 		 657 			 460
5 	 8 	 9 	 162 		 561 			 398


## Poseidon

In [9]:
poseidon_params = [
#[d, n, r_f, r_p]
 [3, 3, 8, 84],
 [5, 3, 8, 56],
 [3, 4, 8, 84],
 [5, 4, 8, 56],
 [3, 5, 8, 84],
 [5, 5, 8, 56],
 [3, 6, 8, 84],
 [5, 6, 8, 56],
 [3, 8, 8, 84],
 [5, 8, 8, 56],
]

print("Poseidon constraints")
print("d", "\t", 
      "n", "\t", 
      "r_f", "\t", 
      "r_p", "\t", 
      "N_R1CS", "\t",
      "N_Plonk_2_wire", "\t", 
      "N_Plonk_3_wire")
for param in poseidon_params:
    d = param[0]
    n = param[1]
    r_f = param[2]
    r_p = param[3]
    print(d, "\t", 
          n, "\t", 
          r_f, "\t", 
          r_p, "\t", 
          poseidon_R1CS_constraints(d, n, r_f, r_p), "\t\t",
          poseidon_Plonk_2_wire_constraints(d, n, r_f, r_p), "\t\t\t",
          poseidon_Plonk_3_wire_constraints(d, n, r_f, r_p))

Poseidon constraints
d 	 n 	 r_f 	 r_p 	 N_R1CS 	 N_Plonk_2_wire 	 N_Plonk_3_wire
3 	 3 	 8 	 84 	 216 		 768 			 492
5 	 3 	 8 	 56 	 240 		 624 			 432
3 	 4 	 8 	 84 	 232 		 1336 			 600
5 	 4 	 8 	 56 	 264 		 1032 			 520
3 	 5 	 8 	 84 	 248 		 2088 			 708
5 	 5 	 8 	 56 	 288 		 1568 			 608
3 	 6 	 8 	 84 	 264 		 3024 			 1368
5 	 6 	 8 	 56 	 312 		 2232 			 1080
3 	 8 	 8 	 84 	 296 		 5448 			 2504
5 	 8 	 8 	 56 	 360 		 3944 			 1896
